In [1]:
import os

In [2]:
os.chdir('../../')
os.system("php artisan migrate:fresh")
os.chdir('./database/transformer/')

In [3]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import numpy as np
import requests as r

In [4]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [5]:
with open('./source/questionnaire.json') as json_file:
    data = json.load(json_file)
    data = data['top_container']

In [6]:
exclude = [43374942,43426483,43374941,43374933,43375020,43374922,43398762]
#exclude = []

In [7]:
country_key = 43380772
funds = 43374826
contribution = 43374827

In [8]:
config = {
    "values":{
        "43374939":"Type of Action",
        "43374932":"Role Organisation",
        "43374862":"Responsible Actor",
        "43375025":"Reporting",
        "43374951":"Reporting and Evaluations",
        "43374934":"Outcome Evaluated",
        "43374949":"Geography",
        "43374915":"Source to Sea",
        "43374916":"Lifecycle of Plastics",
        "43374904":"Target Action",
        "43374931":"Impact",
        "43374917":"Pollutant Targeted",
        "43374905":"Sector",
        "43374920":"Funding",
        "43374943":"Duration",
    },
    "country":{
        "id": "43380772"
    }
}

In [9]:
all_childs = []

def getAllChilds(x, pid=None, p=None):
    qid = None
    text = None
    dependencies = None
    if 'q_no' in x:
        qid = str(x['q_no'])
    if p:
        if 'q_no' in p:
            qid = str(p['q_no'])
        if 'q_no' in x:
            qid = str(x['q_no'])
    if 'text' in x:
        text = x['text']
        if text == " ":
            text = p['text']
    if 'title' in x:
        text = x['title']
    if pid:
        pid = str(pid)
    if 'dependencies' in x:
        dependencies = json.loads(x['dependencies'])
        dependencies = dependencies[0]
        rule = int(dependencies['rule'])
        parent = [o for o in p['options'] if o['value'] == rule]
        pid = str(parent[0]['id'])
    if text:
        text = BeautifulSoup(text)
        text = text.get_text()
        all_childs.append({
            "id": x['id'],
            "name": text.replace('\xa0',' ').replace('\n','').replace('  ',' '),
            "pid": pid,
            "qid": qid,
            "type": x["class"],
        })
    if 'options' in x:
        for z in x['options']:
            getAllChilds(z, x['id'], x)
    if 'children' in x:
        for z in x['children']:
            getAllChilds(z, x['id'], x)

In [10]:
for d in data['children']:
    getAllChilds(d)

In [11]:
df = pd.DataFrame(all_childs)

In [12]:
df = df[~df['type'].isin(['Page','Section','Note'])]

In [13]:
df['qpar'] = df['qid'].apply(lambda x: x.split('.')[0]).astype('int')

In [14]:
dfids = df['id'].to_list()

In [15]:
def replaceParentId(pid):
    if int(pid) not in dfids:
        return None
    return pid

In [16]:
df['pid'] = df['pid'].apply(replaceParentId) 

In [17]:
#resp = pd.read_csv('./source/data-production-fix-narrative-submission-country-names_2020-02-03.csv')
resp = pd.read_csv('./source/data-production_2021-02-25.csv')

In [18]:
def trim_all_columns(df):
    trim_strings = lambda x: x.strip().rstrip() if isinstance(x, str) else x
    return df.applymap(trim_strings)
resp = trim_all_columns(resp)

In [19]:
resp.drop(resp.columns[resp.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [20]:
q_rename = {}
for qs in list(resp):
    qid = qs.split(' ')[0]
    if qid[-1] == ".":
        qid = qid[:-1]
        qname = '. '.join(qs.split('. ')[1:])
        ids = df[(df['qid']==qid)]
        ids = ids[~(ids['type']=='SelectionOption')]
        try:
            if qname in list(ids['name'])[0]:
                qid = list(ids['id'])[0]
        except:
            pass
    q_rename.update({qs:qid})

In [21]:
records = resp.rename(columns=q_rename)

In [22]:
records[country_key] = records[country_key].apply(lambda x: x.replace(';',',') if x == x else x)

In [23]:
# Define new UUID for Narrative Submission
import uuid
def generateUUID():
    randomId = str(uuid.uuid4()).split('-')
    return '999999-{}-{}-{}'.format(randomId[1],randomId[2],randomId[3])
records['Unique'] = records['Unique'].apply(lambda x : x if x == x else generateUUID())

In [24]:
def phase(x):
    if (x == "First Phase"):
        return 1
    if (x == "Second Phase"):
        return 2
    return 0
records['Phase'] = records['When'].apply(phase).astype('int')

In [25]:
records = records.drop(exclude, axis=1)

In [26]:
pd.DataFrame(records[43374842].str.strip().unique(), columns=["organisation_names"]).to_csv('organisationlist.csv')

In [27]:
non_var = ['When','Phase','StartDate','CompletionDate','Unique' ,country_key]
results = []
for a in records.to_dict('records'):
    for b in a:
        if b not in non_var:
            opt = df[(df['pid'] == str(b)) & (df['type'] == 'SelectionOption')]
            values = []
            for c in opt.to_dict('records'):
                if c['name'] in str(a[b]):
                    values.append(c['id'])
            if len(values) == 0:
                values = np.nan
            if values == values:
                results.append({'uuid':a['Unique'],'qid':b,'values':values,'countries':a[country_key],'phase':a['Phase']})

In [28]:
res = pd.DataFrame(results)

In [29]:
def renameCountries(x):
    x = x.replace('Mazambique','Mozambique')
    x = x.replace('Cost Rica','Costa Rica')
    x = x.replace('Cabo Verde','Cape Verde')
    x = x.replace('Sri lanka','Sri Lanka')
    x = x.replace('Solomon islands','Solomon Islands')
    x = x.replace('Democratic Republic of Congo','Democratic Republic of the Congo')
    x = x.replace('United Kingdom of Great Britain & Northern Ireland','U.K. of Great Britain and Northern Ireland')
    x = x.replace('United States of America','United States')
    x = x.replace('United States','United States of America')
    x = x.replace("Cote D'Ivoire",'Côte d’Ivoire')
    x = x.replace('Moldova','Moldova, Republic of')
    x = x.replace('Venezuela','Venezuela (Bolivarian Republic of)')
    x = x.replace('Iran (Islamic Republic of)','Iran  (Islamic Republic of)')
    return x

In [30]:
## Drop Datapoints with no countries
res = res.dropna(subset=['countries'])

In [31]:
#res['countries'] = res['countries'].apply(lambda x: renameCountries(x))

```
Democratic Republic of Congo
Solomon Islands
Cabo Verde
Sri lanka
Cost Rica
Venezuela (Bolivarian Republic of)
Mazambique
```

In [32]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, query
import numpy as np
from app.models import Countries, Groups, Values, CountryGroups, Datapoints, DatapointCountries, DatapointValues, Questions, Answers
from app.connection import engine_url, write_data

In [33]:
engine_url = engine_url()
engine = create_engine(engine_url)
session = sessionmaker(bind=engine)()

In [34]:
countrylist = pd.read_csv('./source/country_shapefile.csv')
countrylist = countrylist.drop_duplicates('Terr_Name')
countrylist = countrylist.rename(columns={'Terr_Name':'name','Color_Code':'code'})
countrylist = countrylist[countrylist['STATUS'] == 'Member State']
countrylist = countrylist[['name','code']]
#countrylist = countrylist[countrylist['code'] != 'XXX']

In [35]:
countrylist = countrylist.append(pd.DataFrame(
    [{'name':'All','code':None},{'name':'Other','code':None}]
), sort=False)

In [36]:
for country in countrylist.to_dict('records'):
    input_data = Countries(country)
    write_data(session, input_data, country, False)

In [37]:
grouplist = pd.read_csv('./source/country_group.csv')

In [38]:
def getGroupCode(g):
    if ('(') in g:
        g = g.split('(')
        return g[1].replace(')','')
    if ('/') in g:
        g = g.split('/')
        return g[0]
    g = g.split(' ')
    if len(g) > 1:
        gs = []
        for s in g:
            gs.append(s[0].upper())
        return ''.join(gs)
    else:
        return g[0]

In [39]:
groups = grouplist.drop_duplicates('name')
groups = groups[['name','code']]
groups['parent_id'] = None
groups['code'] = groups['name'].apply(lambda x: getGroupCode(x))

In [40]:
for groups in groups.to_dict('records'):
    input_data = Groups(groups)
    write_data(session, input_data, groups, False)

In [41]:
def getId(source, table):
    table = session.query(table).filter(table.name == source).first()
    if table:
        return int(table.id)
    return 0

In [42]:
grouplist['country_id'] = grouplist['country'].apply(lambda x: getId(x, Countries))
grouplist['group_id'] = grouplist['name'].apply(lambda x: getId(x, Groups))

In [43]:
grouplist = grouplist[grouplist['country_id'] != 0]
grouplist = grouplist[['country_id','group_id']]

In [44]:
for groups in grouplist.to_dict('records'):
    input_data = CountryGroups(groups)
    write_data(session, input_data, groups, False)

In [45]:
values = df.rename(columns={'id':'code','pid':'parent_code'})
values = values[['name','code','parent_code','type']]
values['code'] = values['code'].fillna("Empty").astype('str')
values['parent_code'] = values['parent_code'].fillna("Empty").astype('str')
#values = values[~values['name'].str.contains('All of the above')]

In [46]:
def getgrandparent(x):
    if x['type'] == "SelectionOption":
        parent = values[values['code'] == str(x['parent_code'])]
        if parent.shape[0] > 0:
            parent = parent.to_dict('records')[0]
            if parent['type'] == "MultipleChoice" and parent['parent_code'] != "Empty":
                return str(parent['parent_code'])
            elif parent['type'] == "MultipleAnswer" and parent['parent_code'] != "Empty":
                return str(parent['parent_code'])
            else:
                return str(parent['code'])
    return str(x['parent_code'])

In [47]:
def setDrop(x):
    if x['type'] in ["FreeText","GridRow","ScaleRow","Scale"]:
        return True
    if x['type'] == "MultipleChoice" and x['parent_code'] != "Empty":
        return True
    if x['type'] == "MultipleAnswer" and x['parent_code'] != "Empty":
        return True
    return False

In [48]:
values['gp'] = values.apply(getgrandparent, axis=1)
values['drop'] = values.apply(setDrop, axis=1)

In [49]:
### FreeText Question (ftqs)
ftqs = values[values['type'] == 'FreeText']

In [50]:
values.drop(values[values['drop']].index,inplace=True)

In [51]:
def renameColumn(x):
    name = x['name']
    try:
        name = config['values'][str(x['code'])]
    except:
        pass
    return name

In [52]:
values['name'] = values.apply(lambda x: renameColumn(x), axis=1)
values = values.reset_index()
values['parent_code'] = values.apply(lambda x: x['gp'] if x['type'] == "SelectionOption" else x['parent_code'],axis=1)
values['id'] = values['code'].astype(int)

In [53]:
includes = []
for a in config['values']:
    includes.append(int(a))

In [54]:
values.drop(values[(~values['id'].isin(includes)) & (values['type'] != "SelectionOption")].index, inplace=True)

In [55]:
allparentids = values['code'].to_list()
allparentids.append("Empty")

In [56]:
values['drop'] = values['parent_code'].apply(lambda x: False if x in allparentids else True)
values.drop(values[values['drop']].index, inplace=True)

In [57]:
values['parent_code'] = values['parent_code'].apply(lambda x: x if x != "Empty" else None)

In [58]:
values = values[['name','code','parent_code','type']]

In [59]:
### Value Table
for value in values.to_dict('records'):
    run = True
    if value['type'] == 'SelectionList' and value['parent_code'] != None:
        run = False
    if run:
        value.update({'parent_id':None,'description':None})
        parent = session.query(Values).filter(Values.code == value['parent_code']).first()
        if parent:
            value.update({'parent_id':parent.id})
        input_data = Values(value)
        write_data(session, input_data, value, False)

In [60]:
### Questions Table
for item in ftqs.to_dict('records'):
    item.update({'parent_id':None, 'value_id': None})
    parent = False
    value = False
    if item['parent_code'] != 'Empty':
        parent = session.query(Questions).filter(Questions.code == item['parent_code']).first()
        value = session.query(Values).filter(Values.code == item['parent_code']).first()
    if parent:
        item.update({'parent_id':parent.id, 'type': item['type']})
    if value:
        item.update({'value_id':value.id, 'type': 'Dependency'})
    input_data = Questions(item)
    write_data(session, input_data, item, False)

In [61]:
### Datapoint Table
for uuid in set(list(records['Unique'])):
    rec = records.fillna(0)
    colval = rec[rec['Unique'] == uuid][[funds, contribution, 'Phase']]
    fund = int(colval.to_dict('records')[0][funds])
    contrib = int(colval.to_dict('records')[0][contribution])
    phase = colval.to_dict('records')[0]['Phase']
    datapoint = {'uuid':uuid,'funds':fund,'contribution':contrib,'phase':phase}
    input_data = Datapoints(datapoint)
    write_data(session, input_data, datapoint, False)

In [62]:
### DatapointCountries Table
unlisted_country = []
for dc in records[['Unique', country_key]].to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == dc['Unique']).first()
    added_country = []
    if dc[country_key] == dc[country_key]:
        for c in dc[country_key].split(','):
            c = c.strip()
            c = renameCountries(c)
            country = session.query(Countries).filter(Countries.name == c).first()
            if country:
                if country.name not in added_country:
                    datapoint_country = {
                        'datapoint_id':datapoint.id,
                        'country_id': country.id
                    }
                    input_data = DatapointCountries(datapoint_country)
                    write_data(session, input_data, datapoint_country, False)
                    added_country.append(country.name)
            else:
                thegroup = session.query(Groups).filter(Groups.name == c).first()
                if thegroup:
                    for cntr in session.query(CountryGroups).filter(CountryGroups.group_id == thegroup.id).all():
                        new_country = session.query(Countries).filter(Countries.id == cntr.country_id).first()
                        if new_country.name not in added_country:
                            datapoint_country = {
                                'datapoint_id': datapoint.id,
                                'country_id': cntr.country_id
                            }
                            input_data = DatapointCountries(datapoint_country)
                            write_data(session, input_data, datapoint_country, False)
                            added_country.append(new_country.name)
                else:
                    if c not in unlisted_country:
                        unlisted_country.append(c)
print(unlisted_country)

['Bolivarian Republic of', 'Republic of', 'Russia', '', 'Anguilla', 'Saint Kitts and Navis']


In [63]:
### DatapointValues Table
for values in res.to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == values['uuid']).first()
    for v in values['values']:
        value = session.query(Values).filter(Values.code == v).first()
        if value:
            datapoint_value = {
                'datapoint_id':datapoint.id,
                'value_id': value.id
            }
            input_data = DatapointValues(datapoint_value)
            write_data(session, input_data, datapoint_value, False)

In [64]:
### Answers Table
qids = []
for x in ftqs['code']:
    if int(x) in list(records.columns):
        qids.append(int(x))
        
for uuid in set(list(records['Unique'])):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == uuid).first()
    rec = records
    colval = rec[rec['Unique'] == uuid][qids]
    for code in list(colval.columns):
        question = session.query(Questions).filter(Questions.code == str(code)).first()
        value = colval[code].values[0]
        answer_value = {
            'datapoint_id': datapoint.id,
            'question_id': question.id,
            'value': [None if value != value else str(value)]
        }
        input_data = Answers(answer_value)
        write_data(session, input_data, answer_value, False)

In [65]:
## DELETE All of the above
d = session.query(Values).filter(Values.name == "All of the above")
d.delete(synchronize_session=False)

7